In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
from file_loading import file_loader, data_types_converter

file_path = "../dataset/"

df_train = file_loader(file_path + "training/leaf.csv.lz4")
df_test = file_loader(file_path + "testing/leaf.csv.lz4")

In [3]:
numerical_data = ['demand', 'destination_current_public_holiday', 'destination_current_school_holiday', 
                  'destination_days_to_next_public_holiday', 'destination_days_to_next_school_holiday', 
                  'od_destination_time', 'od_number_of_similar_12_hours', 
                  'od_number_of_similar_2_hours', 'od_number_of_similar_4_hours', 'od_origin_month', 
                  'od_origin_time', 'od_origin_week', 'od_origin_weekday', 'od_origin_year', 
                  'od_travel_time_minutes', 'origin_current_public_holiday', 'origin_current_school_holiday', 
                  'origin_days_to_next_public_holiday', 'origin_days_to_next_school_holiday', 
                  'price', 'sale_day', 'sale_day_x', 'sale_month', 'sale_week', 'sale_weekday', 'sale_year']

#### Convertion of data types to float when possible

In [4]:
df_train = data_types_converter(df_train, numerical_data)
df_test = data_types_converter(df_test, numerical_data)

df_train_modified = df_train.copy()
df_test_modified = df_test.copy()

In [5]:
print("train :", df_train_modified.shape)
print("test :", df_test.shape)

train : (632841, 31)
test : (32565, 31)


# Creation of dataset to predict on : 
- evaluation is on the sum of all remaining days until departure

In [6]:
days_to_be_used = [-90, -60, -30, -20, -15, -10, -7, -6, -5, -3, -2, -1]
static_features = ['departure_date',
       'destination_current_public_holiday',
       'destination_current_school_holiday',
       'destination_days_to_next_public_holiday',
       'destination_days_to_next_school_holiday', 'destination_station_name',
       'od_destination_time', 'od_number_of_similar_12_hours',
       'od_number_of_similar_2_hours', 'od_number_of_similar_4_hours',
       'od_origin_month', 'od_origin_time', 'od_origin_week',
       'od_origin_weekday', 'od_origin_year', 'od_travel_time_minutes',
       'origin_current_public_holiday', 'origin_current_school_holiday',
       'origin_days_to_next_public_holiday',
       'origin_days_to_next_school_holiday', 'origin_station_name']#13911 elements if groupby on these
fewer_static_features = ['destination_station_name', 'origin_station_name', 'departure_date', 
                         'od_origin_time', 'od_destination_time']#9173 elements if groupby on these
#Differences in terms of size of groupby results would need further data exploration 

changing_features = ['demand', 'price',
       'sale_date', 'sale_day', 'sale_day_x', 'sale_month', 'sale_week',
       'sale_weekday', 'sale_year']

#### groupby realized to create time series based of same travels:
- number of results changes when more (apparently static) features are used?

In [7]:
from data_pretreatment import splitting_travels, extraction_validation_set

dataset_test = splitting_travels(df_test, fewer_static_features)
# print(len(dataset_test), "different travels in the dataset")
complete_preprocessed_dataset, information_on_travel = extraction_validation_set(dataset_test, days_to_be_used)

100%|████████████████████████████████████████████████████████████████████████████████| 417/417 [00:07<00:00, 58.52it/s]


## First test for prediction --> baseline with random forests

In [8]:
features_names = [#'departure_date',
       'destination_current_public_holiday',
       'destination_current_school_holiday',
       'destination_days_to_next_public_holiday',
       'destination_days_to_next_school_holiday',# 'destination_station_name',
       'od_destination_time', 'od_number_of_similar_12_hours',
       'od_number_of_similar_2_hours', 'od_number_of_similar_4_hours',
       'od_origin_month', 'od_origin_time', 'od_origin_week',
       'od_origin_weekday', 'od_origin_year', 'od_travel_time_minutes',
       'origin_current_public_holiday', 'origin_current_school_holiday',
       'origin_days_to_next_public_holiday',
       'origin_days_to_next_school_holiday', 'price', #'origin_station_name', #'sale_date', 
       'sale_day', 'sale_day_x', 'sale_month', 'sale_week',
       'sale_weekday', 'sale_year']

target_train = df_train_modified.demand
features_train = df_train_modified[features_names]

from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(n_estimators=100, max_depth=6)#, random_state=27)
regr.fit(features_train, target_train)

RandomForestRegressor(max_depth=6)

#### Prediction of each day separately

In [9]:
from classical_ml_evaluation import protocol_classical_ML_predictor

real_values = df_test_modified.demand
features_test = df_test_modified[features_names]
prediction_all_days = regr.predict(features_test)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

print("mae :", mean_absolute_error(real_values, prediction_all_days))
print("mse :", mean_squared_error(real_values, prediction_all_days))
print("r2 :", r2_score(real_values, prediction_all_days))

print("std ae :", np.std(np.abs(real_values - prediction_all_days)))

mae : 1.9424429301183095
mse : 20.161594724004118
r2 : 0.6658339286863304
std ae : 4.048272494192937


#### Prediction on the sum of remaining days

In [10]:
from classical_ml_evaluation import protocol_classical_ML_predictor

real_values, prediction_until_travel, information_on_travel_corrected = protocol_classical_ML_predictor(complete_preprocessed_dataset, 
                                                                                                        features_names, regr, 
                                                                                                        information_on_travel)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

print("mae :", mean_absolute_error(real_values, prediction_until_travel))
print("mse :", mean_squared_error(real_values, prediction_until_travel))
print("r2 :", r2_score(real_values, prediction_until_travel))

print("std ae :", np.std(np.abs(np.array(real_values) - np.array(prediction_until_travel))))

mae : 35.7225636523266
mse : 4845.435469710272
r2 : 0.8246162245654879
std ae : 59.74390275011959


### First improvement : categorical features preprocessing

In [11]:
from data_pretreatment import encode_and_bind

cat_features = ['destination_station_name', 'origin_station_name']

for feature in cat_features:
    df_train_modified = encode_and_bind(df_train_modified, feature)
    df_test_modified = encode_and_bind(df_test_modified, feature)

df_train, df_test = df_train.align(df_test, join='left', axis=1)  # inner join

### Second improvement : add of a missing parameter

In [12]:
df_train_modified['od_origin_day'] = pd.to_datetime(df_train_modified.departure_date).dt.day
df_test_modified['od_origin_day'] = pd.to_datetime(df_test_modified.departure_date).dt.day

### Thrird improvement : convertion of cyclic features to adapted ones 

In [13]:
from data_pretreatment import cyclic_features_transform

periodic_features_day = ['od_origin_day', 'sale_day'] 
periodic_features_month = ['od_origin_month', 'sale_month']

df_train_modified = cyclic_features_transform(df_train_modified, periodic_features_day, periodic_features_month)
df_test_modified = cyclic_features_transform(df_test_modified, periodic_features_day, periodic_features_month)

## Second test with transformed features

In [14]:
features_names = [
       'destination_current_public_holiday',
       'destination_current_school_holiday',
       'destination_days_to_next_public_holiday',
       'destination_days_to_next_school_holiday',
       'od_destination_time', 'od_number_of_similar_12_hours',
       'od_number_of_similar_2_hours', 'od_number_of_similar_4_hours',
       'od_origin_month', 'od_origin_time', 'od_origin_week',
       'od_origin_weekday', 'od_origin_year', 'od_travel_time_minutes',
       'origin_current_public_holiday', 'origin_current_school_holiday',
       'origin_days_to_next_public_holiday',
       'origin_days_to_next_school_holiday', 'price', 
       'sale_day', 'sale_day_x', 'sale_month', 'sale_week',
       'sale_weekday', 'sale_year']

target_train = df_train_modified.demand
features_train = df_train_modified[features_names]

from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(n_estimators=100, max_depth=6)#, random_state=27)
regr.fit(features_train, target_train)

RandomForestRegressor(max_depth=6)

#### Each day separetely

In [15]:
from classical_ml_evaluation import protocol_classical_ML_predictor

real_values = df_test_modified.demand
features_test = df_test_modified[features_names]
prediction_all_days = regr.predict(features_test)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

print("mae :", mean_absolute_error(real_values, prediction_all_days))
print("mse :", mean_squared_error(real_values, prediction_all_days))
print("r2 :", r2_score(real_values, prediction_all_days))

print("std ae :", np.std(np.abs(real_values - prediction_all_days)))

mae : 1.9437301706533756
mse : 20.182970668189608
r2 : 0.6654796355172201
std ae : 4.050294272257312


#### Sum of all remaining days

In [16]:
from classical_ml_evaluation import protocol_classical_ML_predictor

real_values, prediction_until_travel, information_on_travel_corrected = protocol_classical_ML_predictor(complete_preprocessed_dataset, 
                                                                                                        features_names, regr, information_on_travel)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

print("mae :", mean_absolute_error(real_values, prediction_until_travel))
print("mse :", mean_squared_error(real_values, prediction_until_travel))
print("r2 :", r2_score(real_values, prediction_until_travel))

print("std ae :", np.std(np.abs(np.array(real_values) - np.array(prediction_until_travel))))

mae : 35.741220368744514
mse : 4846.8009218612815
r2 : 0.8245668011122366
std ae : 59.744172003753846


#### --> No real improvement with features engineering
Final results with classical ML

In [17]:
len(information_on_travel_corrected)

4556

### Third test : xgboost better? --> No

In [18]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

X_train, X_val, y_train, y_val = train_test_split(features_train, target_train, test_size=0.2, random_state=42)

params = {
    # Parameters that we are going to tune.
    'max_depth': 3,
    'min_child_weight': 1,
    'eta':0.01,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:squarederror',
    'eval_metric': 'mae'
}
xgtrain = xgb.DMatrix(np.array(X_train), np.array(y_train))
xgval = xgb.DMatrix(np.array(X_val), np.array(y_val))

model = xgb.train(
    params,
    xgtrain,
    num_boost_round=1000,
    evals=[(xgval, "Validation")],
    early_stopping_rounds=10,
    verbose_eval=False
)
best_iteration = model.best_ntree_limit

real_values, prediction_until_travel, information_on_travel_corrected = protocol_classical_ML_predictor(
    complete_preprocessed_dataset, features_names, regr, information_on_travel)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print("mae :", mean_absolute_error(real_values, prediction_until_travel))
print("mse :", mean_squared_error(real_values, prediction_until_travel))
print("r2 :", r2_score(real_values, prediction_until_travel))

# print("with split :", mean_absolute_error(model.predict(xgtest, ntree_limit=best_iteration), target_test))

mae : 35.741220368744514
mse : 4846.8009218612815
r2 : 0.8245668011122366


# Deep Learning Model

In [19]:
import torch
import torch.nn as nn

import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#### Proposition is to only consider the prices of the last 14 days as inputs for this LSTM, and the demand of the last day as target
==> A more complex approach would require more time

### Normalization of features : necessary for LSTM

In [20]:
from sklearn.preprocessing import MinMaxScaler

scaler_demand = MinMaxScaler(feature_range=(-1, 1))
df_train["scaled_demand"] = scaler_demand.fit_transform(np.array(df_train.demand).reshape(-1, 1))
scaler_price = MinMaxScaler(feature_range=(-1, 1))
df_train["scaled_price"] = scaler_price.fit_transform(np.array(df_train.price).reshape(-1, 1))

### time series separation into multiple dataframes

In [21]:
from data_pretreatment import splitting_travels, extraction_validation_set

dataset_train = splitting_travels(df_train, fewer_static_features)

In [22]:
train_window = 14

In [23]:
from data_pretreatment import create_inout_sequences
train_inout_seq = create_inout_sequences(dataset_train, train_window)

100%|██████████████████████████████████████████████████████████████████████████████| 9173/9173 [02:25<00:00, 62.88it/s]


In [24]:
list_of_tensors = [(torch.FloatTensor(np.array(df[0])).view(-1), torch.FloatTensor(np.array(df[1])).view(-1)) for df in train_inout_seq]
# X_train_data_normalized = torch.stack(list_of_tensors)

In [25]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1, self.hidden_layer_size),
                            torch.zeros(1, self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [26]:
model = LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [27]:
print(model)

LSTM(
  (lstm): LSTM(1, 100)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)


In [28]:
import random

epochs = 100
batch_size = 200

for i in range(epochs):
    batch = random.choices(list_of_tensors, k=batch_size)
    for k in range(4):
    
        for seq, labels in batch:
            optimizer.zero_grad()
            model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                            torch.zeros(1, 1, model.hidden_layer_size))

            y_pred = model(seq)

            single_loss = loss_function(y_pred, labels)
            single_loss.backward()
            optimizer.step()
        
        print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

    #     if i%25 == 1:
        

print(f'epoch: {i:3} loss: {single_loss.item():10.12f}')

epoch:   0 loss: 0.0029091632
epoch:   0 loss: 0.0035382335
epoch:   0 loss: 0.0033164099
epoch:   0 loss: 0.0029336284
epoch:   1 loss: 0.0008304812
epoch:   1 loss: 0.0009595641
epoch:   1 loss: 0.0011312865
epoch:   1 loss: 0.0012801239
epoch:   2 loss: 0.0004098872
epoch:   2 loss: 0.0004176199
epoch:   2 loss: 0.0004445682
epoch:   2 loss: 0.0004723408
epoch:   3 loss: 0.0000027163
epoch:   3 loss: 0.0000024778
epoch:   3 loss: 0.0000154208
epoch:   3 loss: 0.0000355129
epoch:   4 loss: 0.0001068461
epoch:   4 loss: 0.0000700396
epoch:   4 loss: 0.0000423260
epoch:   4 loss: 0.0000155010
epoch:   5 loss: 0.0015244450
epoch:   5 loss: 0.0007985464
epoch:   5 loss: 0.0004090526
epoch:   5 loss: 0.0000390926
epoch:   6 loss: 0.0015436815
epoch:   6 loss: 0.0016332605
epoch:   6 loss: 0.0016940219
epoch:   6 loss: 0.0017512704
epoch:   7 loss: 0.0031987415
epoch:   7 loss: 0.0029019280
epoch:   7 loss: 0.0027565951
epoch:   7 loss: 0.0026516076
epoch:   8 loss: 0.0001389277
epoch:   8

#### Adaptation of the validation dataset to LSTM constraints

In [29]:
from data_pretreatment import create_inout_sequences_validation
all_validation_data = []; all_validation_target = []
for ts in tqdm(complete_preprocessed_dataset):
    available_part, unavailable_part = ts
    if len(available_part) > train_window: #prediction possible
        limit = len(available_part)
        whole_df = pd.concat([available_part.iloc[len(available_part)-train_window:], unavailable_part])
        features_val, target_val = create_inout_sequences_validation(whole_df, train_window, scaler_price)
        all_validation_data.extend(features_val)
        all_validation_target.extend(target_val)

100%|█████████████████████████████████████████████████████████████████████████████| 5004/5004 [00:37<00:00, 133.84it/s]


In [30]:
list_of_tensors_test = [(torch.FloatTensor(np.array(df)).view(-1)) for df in all_validation_data]

### Predictions on validation data --> test on single demand's day each time (no sum until departure)

In [31]:
model.eval()
test_inputs = []
kept_targets = []
for i in tqdm(range(len(list_of_tensors_test))):
    seq = torch.FloatTensor(list_of_tensors_test[i])
    if len(seq)>0:
        with torch.no_grad():
            model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
                            torch.zeros(1, 1, model.hidden_layer_size))
            test_inputs.append(model(seq).item())
            kept_targets.append(all_validation_target[i])

100%|██████████████████████████████████████████████████████████████████████████| 51310/51310 [00:49<00:00, 1032.77it/s]


#### To inverse previous scaling

In [32]:
actual_predictions = scaler_demand.inverse_transform(np.array(test_inputs).reshape(-1, 1))
print(actual_predictions.shape)
print(np.array(kept_targets).shape)

(51310, 1)
(51310,)


#### Results

In [33]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print("mae :", mean_absolute_error(kept_targets, actual_predictions))
print("mse :", mean_squared_error(kept_targets, actual_predictions))
print("r2 :", r2_score(kept_targets, actual_predictions))

mae : 7.946596261423899
mse : 218.45434511714237
r2 : -0.13107101450863845


### Conclusion on deep learning approach : 
- does not work well, no convergence (seen during the training), then performances remain bad with validation